In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd
import numpy as np
import random
import time
import pickle
import json
from random import randint
import re
from bs4 import BeautifulSoup
import requests

!pip install unidecode
from unidecode import unidecode

Mounted at /content/gdrive
     |████████████████████████████████| 235 kB 5.3 MB/s 


In [ ]:
from lxml.html.clean import Cleaner
cleaner = Cleaner(page_structure=True)

# Nova Extração de links

In [ ]:
def headerURLSImoveis(num_page_def, dominio_imob_def, type_user_agent = 1):
  if type_user_agent == 2: user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
  elif type_user_agent == 1: user_agent = 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Mobile Safari/537.36'
  elif type_user_agent == 3: return {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}

  return {
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
      'Accept-Encoding': 'gzip, deflate',
      'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
      'Connection': 'keep-alive',
      'Host': f'{dominio_imob_def}',
      'Origin': f'{dominio_imob_def}',
      'Referer': f'https://{dominio_imob_def}',
      'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
      'sec-ch-ua-mobile': '?0',
      'sec-ch-ua-platform': '"Windows"',
      'Sec-Fetch-Dest': 'document',
      'Sec-Fetch-Mode': 'navigate',
      'Sec-Fetch-Site': 'same-origin',
      'Sec-Fetch-User': '?1',
      'Upgrade-Insecure-Requests': '1',
      'User-Agent': user_agent,
  }

In [ ]:
df_urls = pd.DataFrame(columns=['Imobiliária', 'URL', 'Tipo', 'Situacao', 'Dominio'])

In [ ]:
imobiliarias = []
#imobiliarias.append(({'Imobiliária':'Associadas', 'Palavra Padrao':'imovel/', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, ({'Tipo': {'Apartamento':{'Comprar':['https://www.associadas.com.br/comprar-imoveis/?t=1&d=&c=7742&ba=&pg=[NUMERO_PAGINA]'],
#                                                                              'Alugar':['https://www.associadas.com.br/alugar-imoveis/?t=1&d=&c=7742&ba=&pg=[NUMERO_PAGINA]']},
#                                                            'Casa':{'Comprar':['https://www.associadas.com.br/comprar-imoveis/?t=1&d=&c=7742&ba=&pg=[NUMERO_PAGINA]'],
#                                                                    'Alugar':['https://www.associadas.com.br/alugar-imoveis/?t=1&d=&c=7742&ba=&pg=[NUMERO_PAGINA]']}        }} )))

imobiliarias.append(({'Imobiliária':'Associadas', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.associadas.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, ({'Tipo': {'Apartamento':{'Comprar':['https://www.associadas.com.br/comprar-imoveis//&t=1&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.associadas.com.br/alugar-imoveis//&t=1&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},  
                                                         'Casa':{'Comprar': ['https://www.associadas.com.br/comprar-imoveis//&t=10&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]','https://www.associadas.com.br/comprar-imoveis//&t=15&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.associadas.com.br/alugar-imoveis//&t=10&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]', 'https://www.associadas.com.br/alugar-imoveis//&t=15&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Kitnet':{'Comprar':['https://www.associadas.com.br/comprar-imoveis//&t=2&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.associadas.com.br/alugar-imoveis//&t=2&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},        }} )))

imobiliarias.append(({'Imobiliária':'Ativa', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.ativacorretoradeimoveis.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, ({'Tipo': {'Apartamento':{'Comprar':['https://www.ativacorretoradeimoveis.com.br/comprar-imoveis//&t=1&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.ativacorretoradeimoveis.com.br/alugar-imoveis//&t=1&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},  
                                                         'Casa':{'Comprar': ['https://www.ativacorretoradeimoveis.com.br/comprar-imoveis//&t=10&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]','https://www.ativacorretoradeimoveis.com.br/comprar-imoveis//&t=15&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.ativacorretoradeimoveis.com.br/alugar-imoveis//&t=10&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]', 'https://www.ativacorretoradeimoveis.com.br/alugar-imoveis//&t=15&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Kitnet':{'Comprar':['https://www.ativacorretoradeimoveis.com.br/comprar-imoveis//&t=2&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.ativacorretoradeimoveis.com.br/alugar-imoveis//&t=2&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},        }} )))

imobiliarias.append(({'Imobiliária':'Benvegnú Imóveis', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.benvegnuimoveis.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, ({'Tipo': {'Apartamento':{'Comprar':['https://www.benvegnuimoveis.com.br/comprar-imoveis//&t=1&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.benvegnuimoveis.com.br/alugar-imoveis//&t=1&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.benvegnuimoveis.com.br/comprar-imoveis//&t=10&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]','https://www.benvegnuimoveis.com.br/comprar-imoveis//&t=15&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.benvegnuimoveis.com.br/alugar-imoveis//&t=10&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]', 'https://www.benvegnuimoveis.com.br/alugar-imoveis//&t=15&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Kitnet':{'Comprar':['https://www.benvegnuimoveis.com.br/comprar-imoveis//&t=2&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.benvegnuimoveis.com.br/alugar-imoveis//&t=2&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},        }} )))

imobiliarias.append(({'Imobiliária':'Finardi Carrão', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.finardicarraoimoveis.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, 
                     ({'Tipo': {'Apartamento':{'Comprar':['https://www.finardicarraoimoveis.com.br/comprar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.finardicarraoimoveis.com.br/alugar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.finardicarraoimoveis.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.finardicarraoimoveis.com.br/alugar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']}}})))

In [ ]:
imobiliarias.append(({'Imobiliária':'Imóveis endres', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.endresimoveis.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''},
                                         ({'Tipo': {'Apartamento':{'Comprar':['https://www.endresimoveis.com.br/comprar-imoveis//&t=1&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.endresimoveis.com.br/alugar-imoveis//&t=1&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.endresimoveis.com.br/comprar-imoveis//&t=10&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]','https://www.endresimoveis.com.br/comprar-imoveis//&t=15&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.endresimoveis.com.br/alugar-imoveis//&t=10&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]', 'https://www.endresimoveis.com.br/alugar-imoveis//&t=15&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Kitnet':{'Comprar':['https://www.endresimoveis.com.br/comprar-imoveis//&t=2&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.endresimoveis.com.br/alugar-imoveis//&t=2&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},        }} )))

imobiliarias.append(({'Imobiliária':'Imobiliárias Coligadas', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.coligadas.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''},
                                         ({'Tipo': {'Apartamento':{'Comprar':['https://www.coligadas.com.br/comprar-imoveis/comprar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.coligadas.com.br/alugar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.coligadas.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.coligadas.com.br/alugar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Bellotti', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.bellottiimoveis.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''},
                                         ({'Tipo': {'Apartamento':{'Comprar':['https://www.bellottiimoveis.com.br/comprar-imoveis/comprar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.bellottiimoveis.com.br/alugar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.bellottiimoveis.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.bellottiimoveis.com.br/alugar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Ceolin Imóveis', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.ceolinimoveis.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''},
                                         ({'Tipo': {'Apartamento':{'Comprar':['https://www.ceolinimoveis.com.br/comprar-imoveis/comprar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.ceolinimoveis.com.br/alugar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.ceolinimoveis.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.ceolinimoveis.com.br/alugar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Bins Imobiliária', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.binsimoveis.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''},
                                         ({'Tipo': {'Apartamento':{'Comprar':['https://www.binsimoveis.com.br/comprar-imoveis/comprar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.binsimoveis.com.br/alugar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.binsimoveis.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.binsimoveis.com.br/alugar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']}}})))

In [ ]:
imobiliarias.append(({'Imobiliária':'Medina', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.medinaimoveis.com', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, ({'Tipo': 
                                                      {'Apartamento':{'Comprar':['https://www.medinaimoveis.com/comprar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.medinaimoveis.com/alugar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.medinaimoveis.com/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.medinaimoveis.com/alugar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Espendlor Imóveis', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.esplendorimoveis.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, ({'Tipo': {'Apartamento':{'Comprar':['https://www.esplendorimoveis.com.br/comprar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.esplendorimoveis.com.br/alugar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.esplendorimoveis.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.esplendorimoveis.com.br/alugar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Casa Tua Transações Imobiliárias', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.imobiliariacasatua.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, ({'Tipo': {'Apartamento':{'Comprar':['https://www.imobiliariacasatua.com.br/comprar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.imobiliariacasatua.com.br/alugar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.imobiliariacasatua.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.imobiliariacasatua.com.br/alugar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Aluga Administradora de Bens LTDA', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.alugapf.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, ({'Tipo': {'Apartamento':{'Comprar':['https://www.alugapf.com.br/comprar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.alugapf.com.br/alugar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.alugapf.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.alugapf.com.br/alugar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Gilberto Pedott Corretor de Imóveis', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.pedottimoveis.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, ({'Tipo': {'Apartamento':{'Comprar':['https://www.pedottimoveis.com.br/comprar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.pedottimoveis.com.br/alugar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.pedottimoveis.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.pedottimoveis.com.br/alugar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Cau Imóveis', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.cauimoveis.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, ({'Tipo': {'Apartamento':{'Comprar':['https://www.cauimoveis.com.br/comprar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.cauimoveis.com.br/alugar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.cauimoveis.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.cauimoveis.com.br/alugar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Bertolini e Schneider Imóveis', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.besimoveis.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, ({'Tipo': {'Apartamento':{'Comprar':['https://www.besimoveis.com.br/comprar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.besimoveis.com.br/alugar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.besimoveis.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.besimoveis.com.br/alugar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Graffer Negócios Imobiliários', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.grafferimoveis.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, ({'Tipo': {'Apartamento':{'Comprar':['https://www.grafferimoveis.com.br/comprar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.grafferimoveis.com.br/alugar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.grafferimoveis.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.grafferimoveis.com.br/alugar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Área Nobre Imóveis', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.areanobreimoveis.com', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, ({'Tipo': {'Apartamento':{'Comprar':['https://www.areanobreimoveis.com/comprar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.areanobreimoveis.com/alugar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.areanobreimoveis.com/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.areanobreimoveis.com/alugar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Ellen Negócios Imobiliários', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.ellenimoveis.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, ({'Tipo': {'Apartamento':{'Comprar':['https://www.ellenimoveis.com.br/comprar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.ellenimoveis.com.br/alugar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.ellenimoveis.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.ellenimoveis.com.br/alugar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Formighieri', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.formighieriimoveis.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, ({'Tipo': {'Apartamento':{'Comprar':['https://www.formighieriimoveis.com.br/comprar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.formighieriimoveis.com.br/alugar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.formighieriimoveis.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.formighieriimoveis.com.br/alugar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Costa Lemes Imobiliária', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.costalemes.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, ({'Tipo': {'Apartamento':{'Comprar':['https://www.costalemes.com.br/comprar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.costalemes.com.br/alugar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.costalemes.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.costalemes.com.br/alugar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Xarao', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.xaraoimoveis.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''},
                     ({'Tipo': {'Apartamento':{'Comprar':['https://www.xaraoimoveis.com.br/comprar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.xaraoimoveis.com.br/alugar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.xaraoimoveis.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.xaraoimoveis.com.br/alugar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']}}})))

In [ ]:
imobiliarias.append(({'Imobiliária':'Aporte Imóveis', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.aportepf.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, 
                     ({'Tipo': {'Apartamento':{'Comprar':['https://www.aportepf.com.br/comprar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.aportepf.com.br/alugar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.aportepf.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.aportepf.com.br/alugar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Imobiliária Ariotti', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.ariotti.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, 
                     ({'Tipo': {'Apartamento':{'Comprar':['https://www.ariotti.com.br/comprar-imoveis/apartamentos-passo-fundo/&c=7742&pg=[NUMERO_PAGINA]'],
                                                'Alugar':['https://www.ariotti.com.br/alugar-imoveis/apartamentos-passo-fundo/&c=7742&pg=[NUMERO_PAGINA]']},
                                'Casa':{'Comprar': ['https://www.ariotti.com.br/comprar-imoveis/casas-passo-fundo/&c=7742&pg=[NUMERO_PAGINA]'],
                                                'Alugar':['https://www.ariotti.com.br/alugar-imoveis/casas-passo-fundo/&c=7742&pg=[NUMERO_PAGINA]']}}})))


imobiliarias.append(({'Imobiliária':'Arnel', 'Palavra Padrao':'/imovel/', 'Dominio Imob': 'www.arnelimoveis.com.br', 'TAG':'h1', 'Class':'', 'id':''}, 
                     ({'Tipo': {'Apartamento':{'Comprar':['https://arnelimoveis.com.br/busca/?finalidade=Venda&tipo%5B0%5D=Apartamento&cidade=Passo+Fundo&valor%5B0%5D=&valor%5B1%5D=&area%5B0%5D=&area%5B1%5D=&AreaPrivativa%5B0%5D=&AreaPrivativa%5B1%5D=&listagem=grid&pagina=[NUMERO_PAGINA]'],
                                                'Alugar':['https://arnelimoveis.com.br/busca/?finalidade=Aluguel&tipo%5B0%5D=Apartamento&cidade=Passo+Fundo&valor%5B0%5D=&valor%5B1%5D=&area%5B0%5D=&area%5B1%5D=&AreaPrivativa%5B0%5D=&AreaPrivativa%5B1%5D=&listagem=grid&pagina=[NUMERO_PAGINA]']},
                                'Casa':{'Comprar': ['https://arnelimoveis.com.br/busca/?finalidade=Venda&tipo%5B%5D=Casa&cidade=Passo+Fundo&valor%5B0%5D=&valor%5B1%5D=&area%5B0%5D=&area%5B1%5D=&AreaPrivativa%5B0%5D=&AreaPrivativa%5B1%5D=&listagem=grid&pagina=[NUMERO_PAGINA]', 'https://arnelimoveis.com.br/busca/?finalidade=Venda&tipo%5B0%5D=Casa+em+Condominio&cidade=Passo+Fundo&valor%5B0%5D=&valor%5B1%5D=&area%5B0%5D=&area%5B1%5D=&AreaPrivativa%5B0%5D=&AreaPrivativa%5B1%5D=&listagem=grid&pagina=[NUMERO_PAGINA]'],
                                                'Alugar':['https://arnelimoveis.com.br/busca/?finalidade=Aluguel&tipo%5B0%5D=Casa&cidade=Passo+Fundo&valor%5B0%5D=&valor%5B1%5D=&area%5B0%5D=&area%5B1%5D=&AreaPrivativa%5B0%5D=&AreaPrivativa%5B1%5D=&listagem=grid&pagina=[NUMERO_PAGINA]', 'https://arnelimoveis.com.br/busca/?finalidade=Aluguel&tipo%5B%5D=Casa+em+Condominio&cidade=Passo+Fundo&valor%5B0%5D=&valor%5B1%5D=&area%5B0%5D=&area%5B1%5D=&AreaPrivativa%5B0%5D=&AreaPrivativa%5B1%5D=&listagem=grid&pagina=[NUMERO_PAGINA]']},
                                'Kitnet':{'Comprar': ['https://arnelimoveis.com.br/busca/?finalidade=Venda&tipo%5B0%5D=Kitinete&cidade=Passo+Fundo&valor%5B0%5D=&valor%5B1%5D=&area%5B0%5D=&area%5B1%5D=&AreaPrivativa%5B0%5D=&AreaPrivativa%5B1%5D=&listagem=grid&pagina=[NUMERO_PAGINA]'],
                                                'Alugar':['https://arnelimoveis.com.br/busca/?finalidade=Aluguel&tipo%5B0%5D=Kitinete&cidade=Passo+Fundo&valor%5B0%5D=&valor%5B1%5D=&area%5B0%5D=&area%5B1%5D=&AreaPrivativa%5B0%5D=&AreaPrivativa%5B1%5D=&listagem=grid&pagina=[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Casa Nostra', 'Palavra Padrao':'/imovel/', 'Dominio Imob': 'www.casanostraimobiliaria.com.br', 'TAG':'div', 'Class':'product-filter', 'id':''}, 
                     ({'Tipo': {'Apartamento':{'Comprar':['https://casanostraimobiliaria.com.br/imoveis/negociacao/venda/tipoimovel/2507::apartamento/cidade/passo-fundo/pagina/[NUMERO_PAGINA]'],
                                                'Alugar':['https://casanostraimobiliaria.com.br/imoveis/negociacao/locacao/tipoimovel/2507::apartamento/cidade/passo-fundo/pagina/[NUMERO_PAGINA]']},
                                'Casa':{'Comprar': ['https://casanostraimobiliaria.com.br/imoveis/negociacao/venda/tipoimovel/2508::casa/cidade/passo-fundo/pagina/[NUMERO_PAGINA]'],
                                                'Alugar':['https://casanostraimobiliaria.com.br/imoveis/negociacao/locacao/tipoimovel/2508::casa/cidade/passo-fundo/pagina/[NUMERO_PAGINA]']},
                                'Kitnet':{'Comprar': ['https://casanostraimobiliaria.com.br/imoveis/negociacao/venda/tipoimovel/2517::kitinete/cidade/passo-fundo/pagina/[NUMERO_PAGINA]'],
                                                'Alugar':['https://casanostraimobiliaria.com.br/imoveis/negociacao/locacao/tipoimovel/2517::kitinete/cidade/passo-fundo/pagina/[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Castilhos Nazari', 'Palavra Padrao':'vel/', 'Dominio Imob': 'www.castilhosnazari.com.br', 'TAG':'div', 'Class':'rh_page__head', 'id':''},
                     ({'Tipo': {'Apartamento':{'Comprar':['https://castilhosnazari.com.br/properties-search/page/[NUMERO_PAGINA]/?type%5B0%5D=apartamento&location%5B0%5D=passo-fundo-rs'],
                                                'Alugar':['']},
                                'Casa':{'Comprar': ['https://castilhosnazari.com.br/properties-search/page/[NUMERO_PAGINA]/?type%5B0%5D=casa&location%5B0%5D=passo-fundo-rs'],
                                                'Alugar':['']} }})))

In [ ]:
imobiliarias.append(({'Imobiliária':'Casa Nostra', 'Palavra Padrao':'/imovel/', 'Dominio Imob': 'www.casanostraimobiliaria.com.br', 'TAG':'div', 'Class':'product-filter', 'id':''}, 
                     ({'Tipo': {'Apartamento':{'Comprar':['https://casanostraimobiliaria.com.br/imoveis/negociacao/venda/tipoimovel/2507::apartamento/cidade/passo-fundo/pagina/[NUMERO_PAGINA]'],
                                                'Alugar':['https://casanostraimobiliaria.com.br/imoveis/negociacao/locacao/tipoimovel/2507::apartamento/cidade/passo-fundo/pagina/[NUMERO_PAGINA]']},
                                'Casa':{'Comprar': ['https://casanostraimobiliaria.com.br/imoveis/negociacao/venda/tipoimovel/2508::casa/cidade/passo-fundo/pagina/[NUMERO_PAGINA]'],
                                                'Alugar':['https://casanostraimobiliaria.com.br/imoveis/negociacao/locacao/tipoimovel/2508::casa/cidade/passo-fundo/pagina/[NUMERO_PAGINA]']},
                                'Kitnet':{'Comprar': ['https://casanostraimobiliaria.com.br/imoveis/negociacao/venda/tipoimovel/2517::kitinete/cidade/passo-fundo/pagina/[NUMERO_PAGINA]'],
                                                'Alugar':['https://casanostraimobiliaria.com.br/imoveis/negociacao/locacao/tipoimovel/2517::kitinete/cidade/passo-fundo/pagina/[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Castilhos Nazari', 'Palavra Padrao':'vel/', 'Dominio Imob': 'www.castilhosnazari.com.br', 'TAG':'div', 'Class':'rh_page__head', 'id':''},
                     ({'Tipo': {'Apartamento':{'Comprar':['https://castilhosnazari.com.br/properties-search/page/[NUMERO_PAGINA]/?type%5B0%5D=apartamento&location%5B0%5D=passo-fundo-rs'],
                                                'Alugar':['']},
                                'Casa':{'Comprar': ['https://castilhosnazari.com.br/properties-search/page/[NUMERO_PAGINA]/?type%5B0%5D=casa&location%5B0%5D=passo-fundo-rs'],
                                                'Alugar':['']} }})))

imobiliarias.append(({'Imobiliária':'Casa Moove', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.imobiliariacasamoove.com.br', 
                      'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, 
                     ({'Tipo': {'Apartamento':{'Comprar':['https://www.imobiliariacasamoove.com.br/comprar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                        'Alugar':['https://www.imobiliariacasamoove.com.br/alugar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.imobiliariacasamoove.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.imobiliariacasamoove.com.br/alugar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Fragomeni Imóveis', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.fragomeniimoveis.com.br', 
                      'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, ({'Tipo': {'Apartamento':{'Comprar':['https://www.fragomeniimoveis.com.br/comprar-imoveis/apartamentos-passo-fundo/&c=7742&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.fragomeniimoveis.com.br/alugar-imoveis/apartamentos-passo-fundo/&c=7742&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.fragomeniimoveis.com.br/comprar-imoveis/casas-passo-fundo/&c=7742&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.fragomeniimoveis.com.br/alugar-imoveis/casas-passo-fundo/&c=7742&pg=[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Cau Imóveis', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.cauimoveis.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, ({'Tipo': {'Apartamento':{'Comprar':['https://www.cauimoveis.com.br/comprar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.cauimoveis.com.br/alugar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.cauimoveis.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.cauimoveis.com.br/alugar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Bérgamo Imobiliária', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.bergamoimobiliaria.com.br', 'TAG':'h2', 'Class':'titulo-encontrados titulo-property-h2', 'id':''}, ({'Tipo': {'Apartamento':{'Comprar':['https://www.bergamoimobiliaria.com.br/comprar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.bergamoimobiliaria.com.br/alugar-imoveis/apartamentos-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.bergamoimobiliaria.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.bergamoimobiliaria.com.br/alugar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']}}})))

In [ ]:
imobiliarias.append(({'Imobiliária':'Grupo Imob', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.grupoimob.com.br', 'TAG':'h1', 'Class':'', 'id':'', 'Header':3},
                     ({'Tipo': {'Apartamento':{'Comprar':['https://www.grupoimob.com.br/imoveis/a-venda/apartamento/passo-fundo?pagina=[NUMERO_PAGINA]'],
                                                                        'Alugar':['https://www.grupoimob.com.br/imoveis/para-alugar/apartamento/passo-fundo?pagina=[NUMERO_PAGINA]']},
                                                         'Casa':{'Comprar': ['https://www.grupoimob.com.br/imoveis/a-venda/casa/passo-fundo?&pagina=[NUMERO_PAGINA]'],
                                                                 'Alugar':['https://www.grupoimob.com.br/imoveis/para-alugar/casa/passo-fundo?&pagina=[NUMERO_PAGINA]']}}})))

imobiliarias.append(({'Imobiliária':'Clube House Petrópolis', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.clubhouseimoveis.com.br', 'TAG':'h1', 'Class':'', 'id':'', 'Header':3}, 
                     ({'Tipo': {'Apartamento':{'Comprar':['https://www.clubhouseimoveis.com.br/imoveis/a-venda/apartamento/passo-fundo?pagina=[NUMERO_PAGINA]', 'https://www.clubhouseimoveis.com.br/imoveis/a-venda/apartamento-duplex/passo-fundo?pagina=[NUMERO_PAGINA]', 'https://www.clubhouseimoveis.com.br/imoveis/a-venda/apartamento-garden/passo-fundo?pagina=[NUMERO_PAGINA]'],
                                                'Alugar':['https://www.clubhouseimoveis.com.br/imoveis/para-alugar/passo-fundo?pagina=[NUMERO_PAGINA]']},
                                'Casa':{'Comprar': ['https://www.clubhouseimoveis.com.br/imoveis/a-venda/casa/passo-fundo?pagina=[NUMERO_PAGINA]'],
                                                'Alugar':['']},
                                'Kitnet':{'Comprar': ['https://www.clubhouseimoveis.com.br/imoveis/a-venda/kitnet/passo-fundo?pagina=[NUMERO_PAGINA]'],
                                                'Alugar':['']}}})))

imobiliarias.append(({'Imobiliária':'R. Classic', 'Palavra Padrao':'imovel/', 'Dominio Imob': 'www.rclassic.com.br', 'TAG':'h1', 'Class':'', 'id':'', 'Header':3}, 
                     ({'Tipo': {'Apartamento':{'Comprar':['https://www.rclassic.com.br/imoveis/a-venda/apartamento/passo-fundo?pagina=[NUMERO_PAGINA]'],
                                                'Alugar':['https://www.rclassic.com.br/imoveis/para-alugar/passo-fundo?pagina=[NUMERO_PAGINA]']},
                                'Casa':{'Comprar': ['https://www.rclassic.com.br/imoveis/a-venda/casa/passo-fundo?pagina=[NUMERO_PAGINA]'],
                                                'Alugar':['']},
                                'Kitnet':{'Comprar': ['https://www.rclassic.com.br/imoveis/a-venda/kitnet/passo-fundo?pagina=[NUMERO_PAGINA]'],
                                                'Alugar':['https://www.rclassic.com.br/imoveis/para-alugar/kitnet/passo-fundo?pagina=[NUMERO_PAGINA]']}}})))

In [ ]:
imobiliarias.append(({'Imobiliária':'Fabiane Imóveis', 'Palavra Padrao':'imovel_id/', 'Dominio Imob': 'www.fabianeimoveispf.com.br', 'TAG':'div', 'Class':'js-counterProperties', 'id':'', 'Header':3}, 
                     ({'Tipo': {'Apartamento':{'Comprar':['https://www.fabianeimoveispf.com.br/pesquisa/listar?negocio=1&tipo-imovel=1&page=[NUMERO_PAGINA]'],
                                                'Alugar':['']},
                                'Casa':{'Comprar': ['https://www.fabianeimoveispf.com.br/pesquisa/listar?negocio=1&tipo-imovel=2&page=[NUMERO_PAGINA]'],
                                                'Alugar':['']} }})))

In [ ]:
len(imobiliarias)

37

In [ ]:
def verifyQNTImoveisPerPage(url_cont, standard_url, dominio_imob, type_user_agent=1):
  cont_imoveis_per_page = 0
  r = requests.get(url_cont, headers = headerURLSImoveis(num_page_def = 1, dominio_imob_def = dominio_imob, type_user_agent = type_user_agent))
  soup = BeautifulSoup(r.text, "html5")
  urls = soup.find_all('a')
  list_urls_cont = []
  for url in urls:
    try:
      if standard_url in url['href'] and url['href'] not in list_urls_cont:
        list_urls_cont.append(url['href'])
        cont_imoveis_per_page +=1
    except:pass
  return cont_imoveis_per_page

In [ ]:
print(df_urls.shape)

(0, 5)


In [ ]:
df_urls = pd.DataFrame(columns=['Imobiliária', 'URL', 'Tipo', 'Situacao', 'Dominio'])

In [ ]:
#SÓ EXECUTAR QUANDO INICIAR UMA NOVA EXTRAÇÃO

#df_urls_extracao_parte_2_ok = pd.DataFrame(columns=['URL', 'Situacao', 'Parte'])
#df_urls_extracao_parte_2_ok.to_csv('/content/gdrive/My Drive/NER/DataFrames/URLS/urls_config-26-04.csv', sep='\t')

# Extração URLS Tipo 1

In [ ]:
#df_urls = pd.DataFrame(columns=['Imobiliária', 'URL', 'Tipo', 'Situacao', 'Dominio'])

In [ ]:
name_urls_extract_todos = '/content/gdrive/My Drive/NER/DataFrames/URLS/urls_extract_TODOS-26-04.csv'

In [ ]:
df_urls = pd.read_csv(name_urls_extract_todos, sep='\t')

In [ ]:
df_urls['Imobiliária'].value_counts()

Associadas                1819
Ativa                     1819
Benvegnú Imóveis          1819
Imóveis endres            1819
Imobiliárias Coligadas     485
Finardi Carrão             416
Grupo Imob                  66
Name: Imobiliária, dtype: int64

In [ ]:
# CONFIGURAR O TIPO DE HEADER DE IMOBILIÁRIA DIFERENTE
urls_ok = df_urls['URL'].to_list()
situations = ['Comprar', 'Alugar']
gerar_error = 'opa'
ok = 0
check = 0
for imob in imobiliarias:

  name_imob = imob[0]['Imobiliária']
  #if name_imob != 'Grupo Imob':continue
  if 'coligadas' not in name_imob.lower() and check == 0:continue
  elif 'coligadas' in name_imob.lower() and check == 0:
    check = 1
    #continue
  #if 'ceolin' in name_imob and check == 0:
  #  continue
  #else:check = 1
  
  tag = imob[0]['TAG']
  class_soup = imob[0]['Class']
  id_soup = imob[0]['id']
  try:type_header_imob_info = imob[0]['Header']
  except:type_header_imob_info = 3
  print(tag, class_soup)

  dominio_imob = imob[0]['Dominio Imob']
  types_immobile = [*imob[1]['Tipo'].keys()]

  for type_immobile in types_immobile:
    for situation in situations:
      #if name_imob == 'Imobiliárias Coligadas' and type_immobile == 'Apartamento' or situation == 'Comprar':continue
      print("SITUACAO", situation)
      urls = imob[1]['Tipo'][type_immobile][situation]
      print(urls)
      standard_url = imob[0]['Palavra Padrao']

      for url_extract_link_imob in urls:
        if url_extract_link_imob == '':break
        url_qnt_extract_link_imob = url_extract_link_imob.replace('[NUMERO_PAGINA]', '1')
        print(url_qnt_extract_link_imob)
        number_user_agent = 1
        #if name_imob == 'Fabiane Imóveis':
        #  requests.get(url_qnt_extract_link_imob, headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36'})
        try:
          r = requests.get(url_qnt_extract_link_imob, headers = headerURLSImoveis(num_page_def = 2, dominio_imob_def = dominio_imob, type_user_agent = type_header_imob_info))
          if r.status_code==200:
            number_user_agent = type_header_imob_info
          else:float(gerar_error)
        except:
          try:
            r = requests.get(url_qnt_extract_link_imob, headers = headerURLSImoveis(num_page_def = 2, dominio_imob_def = dominio_imob, type_user_agent = type_header_imob_info))
            if r.status_code==200:
              number_user_agent = type_header_imob_info
            else:float(gerar_error)
          except:
            r = requests.get(url_qnt_extract_link_imob, headers = headerURLSImoveis(num_page_def = 2, dominio_imob_def = dominio_imob, type_user_agent = type_header_imob_info))
            if r.status_code == 200:
              number_user_agent = type_header_imob_info
            else:print("ERROR REQUISIÇÃO")

        print("header user agent", number_user_agent)
        try:
          if tag != '' and class_soup != '':
            qnt_imoveis = int(re.findall('\d+', BeautifulSoup(r.text, "html5").find(tag, class_ = class_soup).text)[0])
          elif tag != '' and id_soup != '':
            qnt_imoveis = int(re.findall('\d+', BeautifulSoup(r.text, "html5").find(tag, id = id_soup).text)[0])
          elif tag != '':
            qnt_imoveis = int(re.findall('\d+', BeautifulSoup(r.text, "html5").find(tag).text)[0])
          else:
            print("VERIFICAR TAGS PARA NUMEROS DE PAGINAS")
            break
        except:
          print("ERROR QNT IMOVEIS")
          #float(gerar_error)
          r = requests.get(url_qnt_extract_link_imob, headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36'})
          if r.status_code == 200:
            number_user_agent = 3
          try:
            if tag != '' and class_soup != '':
              qnt_imoveis = int(re.findall('\d+', BeautifulSoup(r.text, "html5").find(tag, class_ = class_soup).text)[0])
            elif tag != '' and id_soup != '':
              qnt_imoveis = int(re.findall('\d+', BeautifulSoup(r.text, "html5").find(tag, id = id_soup).text)[0])
            elif tag != '':
              qnt_imoveis = int(re.findall('\d+', BeautifulSoup(r.text, "html5").find(tag).text)[0])
            else:
              print("VERIFICAR TAGS PARA NUMEROS DE PAGINAS")
              break
          except:
            print("ERROR QNT IMOVEIS")
            continue

        #if qnt_imoveis == df_urls[(df_urls['Imobiliária'] == name_imob) & (df_urls['Situacao'] == situation)].shape[0]:break

        qnt_imv_per_page = verifyQNTImoveisPerPage(url_qnt_extract_link_imob, standard_url, dominio_imob, type_user_agent = number_user_agent)
        print(qnt_imv_per_page, qnt_imoveis)

        qnt_pages = qnt_imoveis//qnt_imv_per_page
        if qnt_imoveis % qnt_imv_per_page > 0: qnt_pages +=1

        for num_page in range(1,qnt_pages+1):
          url_extract_links_imob = url_extract_link_imob.replace('[NUMERO_PAGINA]', str(num_page))
          print("página", num_page, "de", qnt_pages, " -- ", url_extract_links_imob)
          r = requests.get(url_extract_links_imob, headers = headerURLSImoveis(num_page_def = num_page, dominio_imob_def = dominio_imob, type_user_agent = number_user_agent))
          soup = BeautifulSoup(r.text, "html5")
          urls = soup.find_all('a')
          list_urls_ok = []
          for url in urls:
            try:
              if url['href'] in urls_ok:continue
              if standard_url in url['href'] and url['href'] not in list_urls_ok and 'api.whats' not in url['href']:
                list_urls_ok.append(url['href'])
                #print(url['href'])#urls_config
                url_finish = url['href']
                if dominio_imob.replace('www.','') not in url['href']:
                  if url['href'].strip()[0] == '/':
                    dominio_imob = dominio_imob[:-2] if dominio_imob[-1]=='/' else dominio_imob
                    url_finish = dominio_imob + url['href']
                  elif url['href'].strip()[0] != '/':
                    dominio_imob = dominio_imob if dominio_imob[-1]=='/' else dominio_imob+'/'
                    url_finish = 'https://' + dominio_imob + url['href']
                if 'http' != url_finish[0:4]: 
                  url_finish = 'https://' + url_finish
                #if name_imob == 'Clube House Petrópolis':
                #    url_finish = 'https://www.clubhouseimoveis.com.br'+url['href']
                #elif name_imob == 'R. Classic':
                #    url_finish = 'https://www.rclassic.com.br'+url['href']
                #elif name_imob == 'Grupo Imob':
                #    url_finish = 'https://www.grupoimob.com.br'+url['href']
                print(url_finish)
                df_urls = df_urls.append({'Imobiliária': name_imob, 'URL': url_finish, 'Tipo':type_immobile, 'Situacao': situation, 'Dominio':dominio_imob}, ignore_index = True)
            except:pass
          time.sleep(randint(6,15))
          if num_page % 5 == 0:
            df_urls.to_csv(name_urls_extract_todos, sep='\t')
      df_urls.to_csv(name_urls_extract_todos, sep='\t')
  df_urls.to_csv(name_urls_extract_todos, sep='\t')
  print("save")

#for i in range(df_urls.shape[0]):
#  if df_urls['Imobiliária'].iloc[i] == 'Clube House Petrópolis':
#    df_urls['URL'].iloc[i] = 'https://www.clubhouseimoveis.com.br'+df_urls['URL'].iloc[i]
#  elif df_urls['Imobiliária'].iloc[i] == 'R. Classic':
#    df_urls['URL'].iloc[i] = 'https://www.rclassic.com.br'+df_urls['URL'].iloc[i]
#  elif df_urls['Imobiliária'].iloc[i] == 'Grupo Imob':
#    df_urls['URL'].iloc[i] = 'https://www.grupoimob.com.br'+df_urls['URL'].iloc[i]
    
df_urls.to_csv(name_urls_extract_todos, sep='\t')
print("end")

SITUACAO Comprar
['https://www.imobiliariacasamoove.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=[NUMERO_PAGINA]']
https://www.imobiliariacasamoove.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=1
header user agent 3
24 34
página 1 de 2  --  https://www.imobiliariacasamoove.com.br/comprar-imoveis/casas-passo-fundo/&t=&p=&d=&per=&c=7742&ba=&s=&g=&m=&sm=&or=3&ai=&af=&vi=&vf=&ap=&pg=1
https://www.imobiliariacasamoove.com.br/imovel/venda-casas-02-dormitorio-em-passo-fundo/casa-02-dorm-petropolis/544
https://www.imobiliariacasamoove.com.br/imovel/venda-casas-02-dormitorio-em-passo-fundo/casa-cidade-nova-02-dorm/543
https://www.imobiliariacasamoove.com.br/imovel/venda-casas-03-dormitorio-em-passo-fundo/casa-03-suites/542
https://www.imobiliariacasamoove.com.br/imovel/venda-casas-02-dormitorio-em-passo-fundo/chacara-prox-passo-fundo/541
https://www.imobiliariacasamoove.c

# Tipo 2 - OBS: Site Coratto está fora do ar

In [ ]:
#df_urls = pd.DataFrame(columns=['Imobiliária', 'URL', 'Tipo', 'Situacao', 'Dominio'])

In [ ]:
headers_tipo2 = {
  "accept": "*/*",
  "accept-encoding": "gzip, deflate",
  "accept-language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
  "content-length": "68",
  "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
  "origin": "https://www.jcorattoimoveis.com.br",
  "referer": "https://www.jcorattoimoveis.com.br/pesquisa/listar/",
  "sec-ch-ua": 'Google Chrome";v="95", "Chromium";v="95", ";Not A Brand";v="99',
  "sec-ch-ua-mobile":"?0",
  "sec-ch-ua-platform": "Windows",
  "sec-fetch-dest": "empty",
  "sec-fetch-mode": "empty",
  "sec-fetch-site": "same-origin",
  "user-agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36",
  "x-requested-with": "XMLHttpRequest",
}

In [ ]:
list_types = ['Apartamento', 'Casa']
list_situcao = ['Venda', 'Aluguel']
list_cidades =['Passo Fundo']
id_cidade = 7953
id_situacao = 1
for tipo in list_types:
  number_type = -1
  for name_situacao in list_situcao:
    if name_situacao == 'Venda':id_situacao = 1
    if name_situacao == 'Aluguel':id_situacao = 2
    for cidade in list_cidades:
      if tipo == 'Apartamento':number_type = 1
      elif tipo == 'Casa':number_type = 2
      else:break
      if cidade == 'Passo Fundo':id_cidade = 7953
      print(tipo, number_type, name_situacao)
      params = {'eimovel_tipo_id':number_type,'eimovel_tipo_sub_id':"",'eimovel_transacao_id': id_situacao,'eimovel_uso_id':"", 'evalor_minimo':"",
              'evalor_maximo':"",'ecidade_id': id_cidade,'edormitorios':"",'esuites':"",'ebanheiros':"",'egaragens':""}

      session = requests.Session()
      #print(session.cookies.get_dict())
      respone = session.post('https://www.jcorattoimoveis.com.br/pesquisa/listar', data=params, headers=headers_tipo2)
      cookies = session.cookies.get_dict()

      soup = BeautifulSoup(respone.text)
      imv_find = int(re.findall('\d+',soup.find(id='pesquisa_imoveis_encontrados').text)[0])
      paginas = imv_find//6
      #print(paginas)

      for pagina in range(paginas+1+1):
        print("Página", pagina, "de", paginas)
        data1 = {'paginacao_ajax[pagina]': pagina,'paginacao_ajax[quantidade_imovel]': '6'}

        r2 = requests.post('https://www.jcorattoimoveis.com.br/sites/1/9/_ajax/imoveis_imoveis_pesquisa_listar.php', data=data1, cookies=cookies)
        soup_imvs = BeautifulSoup(r2.text, "html5")

        urls = soup_imvs.find_all('a')
        list_urls_temp = []
        for url in urls:
          try:
            if 'imovel_id' in url['href']:
              if 'https://www.jcorattoimoveis.com.br/'+url['href'] not in list_urls_temp:
                list_urls_temp.append('https://www.jcorattoimoveis.com.br/'+url['href'])
                print('https://www.jcorattoimoveis.com.br/'+url['href'])
                df_urls = df_urls.append({'Imobiliária': 'Coratto', 'URL': url['href'], 'Tipo':type_immobile, 'Situacao': situation, 'Dominio':dominio_imob}, ignore_index = True)
          except:pass
        time.sleep(randint(6,15))
      df_urls.to_csv(name_urls_extract_todos, sep='\t')
df_urls.to_csv(name_urls_extract_todos, sep='\t')

# Tipo 3

In [ ]:
#df_urls = pd.DataFrame(columns=['Imobiliária', 'URL', 'Tipo', 'Situacao', 'Dominio'])

In [ ]:
df_urls.shape

(17802, 6)

In [ ]:
import requests
false = 'false'
list_urls = ['http://bpnegociosimobiliarios.com.br', 'http://corretoracamargoimoveis.com.br', 'http://martinsimoveispf.com.br', 'https://tedescoimoveis.com.br', 'http://carlottoimoveis.com.br', 'http://imoveisrosanalorenzoni.com.br']
for url in list_urls:
  url_search = url
  dominio_imob = url
  if url == 'http://bpnegociosimobiliarios.com.br': name_imob = 'BP Negócios imobiliários'
  elif url == 'http://corretoracamargoimoveis.com.br': name_imob = 'Corretora de Imóveis Camargo'
  elif url == 'http://martinsimoveispf.com.br': name_imob = 'Martins Imóveis'
  elif url == 'https://tedescoimoveis.com.br': name_imob = 'Tedesco Imóveis'
  elif url == 'http://carlottoimoveis.com.br': name_imob = 'Carlotto Imóveis'
  elif url == 'http://imoveisrosanalorenzoni.com.br': name_imob = 'Rosana Lorenzoni Corretora de imóveis'
  else:
    print("OPAAA")
    break
  operacoes = [1, 2]
  types_imvs = [2, 5]
  for op in operacoes:
    if op == 1: operacao = 'Comprar'
    elif op == 2: operacao = 'Alugar'
    for type_imv in types_imvs:
      if type_imv == 2: tipo = 'Casa'
      elif type_imv == 5: 
        if url == 'http://corretoracamargoimoveis.com.br':
          type_imv = 6
        tipo = 'Apartamento'
      print(tipo)
      params = {'bairros': [],'cidade': '4154','codigo': [],'destaque': 'false','empreendimento': "",'exclusivo': 'false','filtropais': 'false','imos_codigo': "",'imoveisBusca': [],'imoveisRows': [],
      'limit': '9','operacao': op,'order': "maxval",'page': 0,'pais': 1,'subtipoimovel': [],'tipoimovel': type_imv,'vlfim': "",'vlini': ""}

      #params={"operacao":op,"codigo":[],"tipoimovel":type_imv,"imos_codigo":"","subtipoimovel":[],"empreendimento":"","destaque":false,"vlini":"","vlfim":"","exclusivo":false,"cidade":"4154","pais":1,
      #        "filtropais":false,"order":"maxval","limit":9,"page":1,"imoveisRows":[],"imoveisBusca":[],"bairros":[]}
      response = requests.post(f'{url_search}/imovel-gets/getImovel/', data=params)
      json = response.json()
      try:
        qnt_imv_search = int(json[0]['rowcount'])
      except:
        #typeExcec(type_imv)
        print("OPAAAAAAAAAAAAAA ERROR")
        continue
      if qnt_imv_search % 9 > 0 and type_imv !=3 and type_imv != 6:
        qnt_imv_per_req = qnt_imv_search // 9 + 1
      else:
        qnt_imv_per_req = qnt_imv_search // 9
      for num_page in range(0,qnt_imv_per_req): #Começa com 0
        print("Página", num_page)
        params = {'bairros': [],'cidade': '4154','codigo': [],'destaque': 'false','empreendimento': "",'exclusivo': 'false','filtropais': 'false','imos_codigo': "",'imoveisBusca': [],'imoveisRows': [],
        'limit': '9','operacao': op,'order': "maxval",'page': num_page,'pais': 1,'subtipoimovel': [],'tipoimovel': type_imv,'vlfim': "",'vlini': ""}
        response = requests.post(f'{url_search}/imovel-gets/getImovel/', data=params)
        json = response.json()
        qnt_imv_search = int(json[0]['rowcount'])
        for i_json in range(len(json)):
          tipo = json[i_json]['tic_nome']
          cod = json[i_json]['imo_codigo']
          #print(tipo,cod)
          tic_nome = ''
          ist_nome = ''
          try:ist_nome ='-'+"-".join(json[i_json]['ist_nome'].split())
          except:ist_nome = ''
          try:url_completed = url_search + '/imovel/' + str(cod)+'/'+unidecode(json[i_json]['tic_nome'] + ist_nome).lower()
          except:url_completed = ''
          url_partial = url_search + '/imovel/' + str(cod)
          print(url_partial)
          df_urls = df_urls.append({'Imobiliária': name_imob, 'URL': url_partial, 'Tipo':tipo, 'Situacao': operacao, 'Dominio':dominio_imob}, ignore_index = True)
        time.sleep(randint(3,7))
      df_urls.to_csv(name_urls_extract_todos, sep='\t')
df_urls.to_csv(name_urls_extract_todos, sep='\t')

# Tipo 4

In [ ]:
#df_urls = pd.DataFrame(columns=['Imobiliária', 'URL', 'Tipo', 'Situacao', 'Dominio'])

In [ ]:
df_urls.shape

(18464, 6)

In [ ]:
list_imobs = [{'Name Imob':'Verdi', 'Dominio': 'https://verdiimoveis.com.br'}]
list_imobs.append({'Name Imob':'Predial', 'Dominio': 'https://predialpf.com.br'})

finalidades = ['venda', 'aluguel']
tipos_residencial = ['apartamento', 'casa', 'casa-em-condominio']
for info_imob in list_imobs:
  name_imob = info_imob['Name Imob']
  dominio_imob = info_imob['Dominio']
  for finalidade in finalidades:
    for tipo in tipos_residencial:
      situation, type_immobile = '', ''
      if tipo == 'apartamento': type_immobile = 'Apartamento'
      elif tipo == 'casa' or tipo == 'casa-em-condominio': type_immobile = 'Casa'

      if finalidade == 'venda':situation = 'Comprar'
      elif finalidade == 'aluguel':situation = 'Alugar'

      payload = {'categoriagrupo': 'Residencial','finalidade': finalidade,'tipo_residencial[]': tipo,'cidadebairro[]': 'passo-fundo','codigo': '','ordenar': 'precoAsc','pagina': '1'}

      session = requests.Session()
      response = session.post(f'{dominio_imob}/{finalidade}/{tipo}/passo-fundo/', data = payload, headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'})
      cookies = session.cookies.get_dict()
      try:
        qnt_imvs = int(re.findall('\d+', BeautifulSoup(response.text, "html5").find('h3', class_='result-totals-phrase').text)[0])
      except:
        print("NÂO TEM IMÓVEIS",f'{dominio_imob}/{finalidade}/{tipo}/passo-fundo/' )
        continue

      qnt_pages = qnt_imvs // 12
      if qnt_imvs % 12 > 0:
        qnt_pages +=1

      list_urls_unique = []
      for num_page in range(1,qnt_pages+1):
        print(num_page, "de", qnt_pages)
        print(f'{dominio_imob}/u-sr.php?page='+str(num_page))
        r2 = session.get(f'{dominio_imob}/u-sr.php?page='+str(num_page), params = {'page': num_page}, cookies=cookies)
        cookies = session.cookies.get_dict()

        soup_imvs = BeautifulSoup(r2.text, "html5")
        list_urls = soup_imvs.find_all('a')
        for url in list_urls:
          if url['href'] in df_urls['URL'].to_list():continue
          if url['href'] in list_urls_unique:continue
          list_urls_unique.append(url['href'])
          print(url['href'])
          df_urls = df_urls.append({'Imobiliária': name_imob, 'URL': url['href'], 'Tipo':type_immobile, 'Situacao': situation, 'Dominio':dominio_imob}, ignore_index = True)
        time.sleep(randint(2,7))
    df_urls.to_csv(name_urls_extract_todos, sep='\t')
  df_urls.to_csv(name_urls_extract_todos, sep='\t')
df_urls.to_csv(name_urls_extract_todos, sep='\t')
print("end")

1 de 2
https://predialpf.com.br/u-sr.php?page=1
https://predialpf.com.br/imovel/1549/apartamento-1-quarto-santa-maria-passo-fundo/
https://predialpf.com.br/imovel/2262/apartamento-1-quarto-centro-passo-fundo/
https://predialpf.com.br/imovel/2270/apartamento-3-quartos-centro-passo-fundo/
https://predialpf.com.br/imovel/1550/apartamento-1-quarto-santa-maria-passo-fundo/
https://predialpf.com.br/imovel/2277/apartamento-2-quartos-santa-maria-passo-fundo/
2 de 2
https://predialpf.com.br/u-sr.php?page=2
1 de 1
https://predialpf.com.br/u-sr.php?page=1
NÂO TEM IMÓVEIS https://predialpf.com.br/aluguel/casa-em-condominio/passo-fundo/


In [ ]:
df_urls.shape

# Tipo 5

In [ ]:
#df_urls = pd.DataFrame(columns=['Imobiliária', 'URL', 'Tipo', 'Situacao', 'Dominio'])

In [ ]:
df_urls.shape

(19363, 6)

In [ ]:
headers_tipo5 = {
    'authority': 'www.bortoliniimoveis.com.br',
    'method': 'POST',
    'path': '/imoveis/RetornaImoveis',
    'scheme': 'https',
    'accept': 'text/html, */*; q=0.01',
    'accept-encoding': 'gzip, deflate',
    'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'content-length': '59',
    'content-type': 'application/json; charset=UTF-8',
    'origin': 'https://www.bortoliniimoveis.com.br',
    'referer': 'https://www.bortoliniimoveis.com.br/comprar/todos/RS/passo-fundo?',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36',
}

In [ ]:
"""Esse bloco é responsável para pegar os links, dos imóveis que são retornados a cada requisição. Além disso, também está retornando 
  os cômodos, quando tem, da parte do front(quando é exibido para escolher no site, não quando abre todo o link do imóvel)
  """
#OBS: fazer a extração pelo filtro da própria página -> comprar (apartamento, casa, kitnet)
list_type = ['Comprar', 'Alugar']
#precisa configurar a chave antes --------------------------#########################----------------------
"""Define as chaves necessárias para colocar na requisição e poder carregar as informações necessárias. Essas chaves são definidas
    e visível na hora em que é feito uma requisção qualquer na imobiliária com o filtro que quiser. Nela, está a chave, em headers.
 """
list_chave = ['38d12ab0-bd20-4fb3-adf2-0551227afa63','08786084-2e3a-4600-837f-11d602aea7c4']
for situacao in list_type:
  #for chave in list_chave:
    if situacao == 'Comprar': chave = list_chave[0]
    if situacao == 'Alugar': chave = list_chave[1]
    num_page = 0
    cont_zeros = 0
    """Quando a contagem de zeros na lógica for maior que 3, então o programa para, pois não há mais nada retornando de informações.
    Essa foi a lógica encontrada para o fim do script"""
    while cont_zeros < 3:
      print(num_page)
      try:
        num_page +=1
        #response = requests.post("https://www.bortoliniimoveis.com.br/imoveis/RetornaImoveis", {"chave":chave,"pagina":num_page}, headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36'})
        response = requests.post("https://www.bortoliniimoveis.com.br/imoveis/RetornaImoveis", {"chave":chave,"pagina":num_page}, headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36'})
        soup = BeautifulSoup(response.text)
      except:continue
      try:
        len_imobs = len(soup.find_all(class_='imovel__item'))
      except:pass
      if len_imobs == 0:
        cont_zeros +=1
        continue
      for i in range(len_imobs):
        try:
          url_imv = 'https://www.bortoliniimoveis.com.br'+soup.find_all(class_='imovel__item')[i].find(class_='item').find('a')['href']
        except:continue
        try:tipo = soup.find_all(class_='imovel__item')[i].find(class_='imovel__item__valor').find(class_='tipo').text.strip()
        except:tipo = 'NENHUM'
        #print(tipo)
        if 'comercial' in tipo.lower():continue
        if 'casa' in tipo.lower() or 'apartament' in tipo.lower() or 'kit' in tipo.lower() or 'quit' in tipo.lower():
          print(url_imv)
          df_urls = df_urls.append({'Imobiliária': 'Bortolini', 'URL': url_imv, 'Tipo':tipo, 'Situacao': situacao, 'Dominio':'https://www.bortoliniimoveis.com.br'}, ignore_index = True)
      if i%50 == 0:time.sleep(4)
      time.sleep(randint(2,6))
    df_urls.to_csv(name_urls_extract_todos, sep='\t')
df_urls.to_csv(name_urls_extract_todos, sep='\t')
print("end")

24
https://www.bortoliniimoveis.com.br/imovel/alugar/apartamento-em-passo-fundo/14377
https://www.bortoliniimoveis.com.br/imovel/alugar/apartamento-em-passo-fundo/14362
https://www.bortoliniimoveis.com.br/imovel/alugar/apartamento-em-passo-fundo/10063
https://www.bortoliniimoveis.com.br/imovel/alugar/apartamento-em-passo-fundo/5163
https://www.bortoliniimoveis.com.br/imovel/alugar/apartamento-em-passo-fundo/21891
25
https://www.bortoliniimoveis.com.br/imovel/alugar/apartamento-em-passo-fundo/12986
https://www.bortoliniimoveis.com.br/imovel/alugar/apartamento-em-passo-fundo/9738
https://www.bortoliniimoveis.com.br/imovel/alugar/apartamento-em-passo-fundo/17336
https://www.bortoliniimoveis.com.br/imovel/alugar/apartamento-em-passo-fundo/1728
26
https://www.bortoliniimoveis.com.br/imovel/alugar/apartamento-em-passo-fundo/9968
https://www.bortoliniimoveis.com.br/imovel/alugar/apartamento-em-passo-fundo/4880
https://www.bortoliniimoveis.com.br/imovel/alugar/apartamento-em-passo-fundo/21193


In [ ]:
df_urls.shape

(21109, 6)

# Tipo 6

In [ ]:
#df_urls = pd.DataFrame(columns=['Imobiliária', 'URL', 'Tipo', 'Situacao', 'Dominio'])

In [ ]:
df_urls.shape

(19161, 6)

In [ ]:
def headerTipo6(referer_tipo):
  if referer_tipo == 1:referer = 'https://www.remax.com.br/'
  elif referer_tipo == 2: referer = 'https://www.remax.com.br/busca.php'
  return {
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
  'Accept-Encoding': 'gzip, deflate',
  'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
  'Cache-Control': 'max-age=0',
  'Connection': 'keep-alive',
  'Content-Length': '180',
  'Content-Type': 'application/x-www-form-urlencoded',
  'Host': 'www.remax.com.br',
  'Origin': 'https://www.remax.com.br',
  'Referer': referer,
  'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"',
  'Sec-Fetch-Dest': 'document',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-User': '?1',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36'
  }

In [ ]:
list_imobs = [{'Name Imob':'Remax', 'Dominio': 'https://www.remax.com.br'}]
list_types_immobile = ['Apartamento', 'Casa']
list_situation_immobile = ['Comprar', 'Alugar']

for info_imob in list_imobs:
  name_imob = info_imob['Name Imob']
  dominio_imob = info_imob['Dominio']
  for type_immobile in list_types_immobile:
    if type_immobile == 'Apartamento':tipoImovelPayload = 1
    elif type_immobile == 'Casa':tipoImovelPayload = 2
    for situation in list_situation_immobile:
      if situation == 'Comprar': pretensaoPayload = "comprar"
      elif situation == 'Alugar': pretensaoPayload = "alugar"
      payload = {"quantidadeImoveis": 15,"opcao": "busca","lingua": "pt","pais": "BR","estado": "","cidade": 7851,"bairro": "",
      "bairros": "","cobertura": "","ordenacao": "bairroOrder","pretensao": pretensaoPayload,"tipoImovel[]": tipoImovelPayload,"termo": "Passo Fundo - RS"
      }
      response = requests.post('https://www.remax.com.br//busca.php', data = payload, headers = headerTipo6(referer_tipo = 1))
      try:
        qnt_imoveis = int(re.findall('\d+', BeautifulSoup(response.text).find(class_='text-resul-seach').text)[0])
      except:
        continue
        print("ERROR QNT IMOVEIS")

      standard_url = 'imovel/'
      list_urls_unique = []
      true = True

      qnt_pages = qnt_imoveis // 15
      if qnt_imoveis % 15 > 0: qnt_pages +=1

      for num_page in range(0,qnt_pages):
        print(num_page, "de", qnt_pages-1, "-", type_immobile, situation)
        #payload_search = {
        #    "filtro": {"tipoImovel":";"+str(tipoImovelPayload),"idCidade":7851,"statusImovelStr":"V","novos":true,"usados":true,"quantidadeImoveis":15,"paginado":true,"startpag":0,"pagAtual":num_page,"ordenacao":"bairroOrder"},
        #      'ordenacao': 'bairroOrder','cobertura': '','buscaInterna': 'true',
        #      'passadorInit': 15,'passadorNum': num_page+1,'quantidadeNum': 15,'qtdTotal': qnt_imoveis,'tipoImovel': '','novos': 'true','usados': 'true','quantidadeImoveis': '15','rede': '','temporada': '',
        #      'cidade': 7851,'bairros': '','pretensao': pretensaoPayload,'tipoImovel[]': tipoImovelPayload,'precoDe': '','precoAte': '','quartosDe': '','quartosAte': '','areaDe': '','areaAte': '',
        #      'termo': 'Passo Fundo - RS'
        #  }
        payload_search = {
            "filtro": {"tipoImovel":";"+str(tipoImovelPayload),"idCidade":7851,"statusImovelStr":"V","novos":'true',"usados":'true',"quantidadeImoveis":15,"paginado":true,"startpag":15*num_page-15,
                       "pagAtual":num_page,"ordenacao":"bairroOrder"},'ordenacao': 'bairroOrder','cobertura': '','buscaInterna': 'true',
              'passadorInit': 15*(num_page+1)-15,'passadorNum': num_page+1,'quantidadeNum': 15,'qtdTotal': qnt_imoveis,'tipoImovel': '','novos': 'true','usados': 'true','quantidadeImoveis': 15,
              'rede': '','temporada': '',
              'cidade': 7851,'bairros': '','pretensao': pretensaoPayload,'tipoImovel[]': tipoImovelPayload,'precoDe': '','precoAte': '','quartosDe': '','quartosAte': '','areaDe': '','areaAte': '',
              'termo': 'Passo Fundo - RS'
          }
        r = requests.post('https://www.remax.com.br/busca.php', data = payload_search, headers = headerTipo6(referer_tipo = 2))
        list_urls = BeautifulSoup(r.text).find_all('a')
        for url in list_urls:
          if url['href'] in list_urls_unique:continue
          if standard_url in url['href']:
            list_urls_unique.append(url['href'])
            print(url['href'])
            df_urls = df_urls.append({'Imobiliária': name_imob, 'URL': url['href'], 'Tipo':type_immobile, 'Situacao': situation, 'Dominio':dominio_imob}, ignore_index = True)
        time.sleep(randint(1,5))
      df_urls.to_csv(name_urls_extract_todos, sep='\t')
    df_urls.to_csv(name_urls_extract_todos, sep='\t')
df_urls.to_csv(name_urls_extract_todos, sep='\t')
print("end")

0 de 5 - Apartamento Comprar
https://www.remax.com.br/imovel/apartamentos-comprar-vender-petropolis-passo-fundo-rio-grande-do-sul/250853
https://www.remax.com.br/imovel/apartamentos-comprar-vender-centro-passo-fundo-rio-grande-do-sul/250859
https://www.remax.com.br/imovel/apartamentos-comprar-vender-sao-cristovao-passo-fundo-rio-grande-do-sul/250866
https://www.remax.com.br/imovel/apartamentos-comprar-vender-vila-santa-maria-passo-fundo-rio-grande-do-sul/250893
https://www.remax.com.br/imovel/apartamentos-comprar-vender-sao-cristovao-passo-fundo-rio-grande-do-sul/354894
https://www.remax.com.br/imovel/apartamentos-comprar-vender-cidade-nova-passo-fundo-rio-grande-do-sul/414285
https://www.remax.com.br/imovel/apartamentos-comprar-vender-petropolis-passo-fundo-rio-grande-do-sul/414294
https://www.remax.com.br/imovel/apartamentos-comprar-vender-centro-passo-fundo-rio-grande-do-sul/418813
https://www.remax.com.br/imovel/apartamentos-comprar-vender-lucas-araujo-passo-fundo-rio-grande-do-sul